## Will it Fit?  
**A container problem solved.**

The containerstore.com has a large selection of containers, but does not provide a search option to specify the size of the container.  My cat clawed the wicker baskets that fit perfectly in my small dresser, so I am in need of replacement containers in a very specific size.  Then I will be able to avoid trashing the entire piece of furniture.

First I scraped the data from the container store website collecting the name, dimensions, and url for each piece.  In this notebook we will be able to search for containers within a given range of appropriate dimensions.


In [332]:
import scrapy
import json
from pprint import pprint

Let's start by looking at the collected data.  

In [333]:
with open('container3.json') as data_file:    
    data = json.load(data_file)

# pprint(data[0:10])
# count = 0
for i in range(0,len(data)):

    if((data[i]['dimensions'])==[]):
        print('no dimensions given:')
        print (data[i]['url'])
#         count += 1
pprint(len(data))
# print(count)

no dimensions given:
https://www.containerstore.com/s/storage/plastic-bins-baskets/white-compact-plastic-bins-4-pack-with-grey-lids/12d?productId=11006520
no dimensions given:
https://www.containerstore.com/s/storage/plastic-bins-baskets/white-compact-plastic-bins-4-pack-with-white-lids/12d?productId=11006211
no dimensions given:
https://www.containerstore.com/s/storage/decorative-bins-baskets/folding-wire-storage-basket/12d?productId=11000564
262


We scraped the Container Store website and found 210 items from the "decorative bins and baskets" section and 52 from the "plastic bins and baskets" section.  Above we print links to the three items do not have dimensions given.  (Scraped on July 20, 2017)

In [334]:
for i in range (15,17):
    pprint(data[i])

{'dimensions': ['13" sq. x 11" h',
                '13" sq. x 11" h',
                '13" sq. x 11" h',
                '13" sq. x 11" h'],
 'price': ['9.99', '9.99', '9.99', '9.99'],
 'title': 'Authentic Milk Crate',
 'url': 'https://www.containerstore.com/s/storage/plastic-bins-baskets/authentic-milk-crate/12d?productId=10000531'}
{'dimensions': ['14" x 18-1/4" x 8-1/2" h', '11" x 15" x 6-1/4" h'],
 'price': ['34.99', '21.99'],
 'title': 'Nova Woven Storage Bins',
 'url': 'https://www.containerstore.com/s/storage/decorative-bins-baskets/nova-woven-storage-bins/12d?productId=11006165'}


Next we adjust the format of the dimensions to make them searchable. They will be listed as [length, width, height] all in inches. 

In [335]:
from fractions import Fraction
# loop through the container links
for j in range (0,len(data)):
    data[j]['new dimensions']=[]
    # many container links have containers available in several different sizes, so we roam through each one
    for k in range(0,len(data[j]['dimensions'])):
        mydim = data[j]['dimensions'][k].split('x')
        newdim = []
        # each dimension of the container needs to be converted to a float and added to 'new dimensions'
        for i in range (0,len(mydim)):
            mysplit = mydim[i].split('sq.')[0]
            if '-' in mysplit:
                this_split = mysplit.split('-')
                num =0
                for p in this_split:
                    num += float(Fraction(p.split('"')[0]))
            else:
                num = float(Fraction(mysplit.split('"')[0]))
            # if the entry notes that it is square or a diameter, then the dimension needs to be inserted in two spots            
            double=int(('sq.' in mydim[i]) or ('diam.' in mydim[i]))
            for _ in range(0,double+1):
                newdim.append(num)
        data[j]['new dimensions'].append(newdim)

In [337]:
for j in range (0,2):
    print('')
    pprint(data[j])
    print(data[j]['dimensions'])
    print(data[j]['new dimensions'])
    


{'dimensions': ['18" x 12-3/4" x 9-7/8" h'],
 'new dimensions': [[18.0, 12.75, 9.875]],
 'price': ['13.99'],
 'title': 'Clear Stackable Plastic Basket with Handles',
 'url': 'https://www.containerstore.com/s/storage/plastic-bins-baskets/clear-stackable-plastic-basket-with-handles/12d?productId=11006058'}
['18" x 12-3/4" x 9-7/8" h']
[[18.0, 12.75, 9.875]]

{'dimensions': ['12" x 9-1/4" x 7" h', '13-7/8" x 10-7/8" x 7-3/4" h'],
 'new dimensions': [[12.0, 9.25, 7.0], [13.875, 10.875, 7.75]],
 'price': ['14.99', '17.99'],
 'title': 'Stone Woven Plastic Storage Bins with Handles',
 'url': 'https://www.containerstore.com/s/storage/plastic-bins-baskets/stone-woven-plastic-storage-bins-with-handles/12d?productId=11005640'}
['12" x 9-1/4" x 7" h', '13-7/8" x 10-7/8" x 7-3/4" h']
[[12.0, 9.25, 7.0], [13.875, 10.875, 7.75]]


The will_it_fit function will be given 6 values, 2 for each dimension (a lower and upper limit).  It will return the possible containers.

In [338]:
def will_it_fit(f1a,f1b,f2a,f2b,f3a,f3b):

    possibles = []
    for j in range(0,len(data)):
        num_of_containers  = len(data[j]['new dimensions'])
        for k in range(0,num_of_containers):
            if (len(data[j]['new dimensions'][k])==3):
                
                if ((f1a<data[j]['new dimensions'][k][0]<f1b) and (f2a<data[j]['new dimensions'][k][1]<f2b) 
                    and (f3a<data[j]['new dimensions'][k][2]<f3b)):

                    possibles.append([data[j],k])

    return possibles
    

The container that I am looking for should be in the range of 6-8.25" x 9-12" x 5-7"

In [339]:
# f1a = 6.
# f1b = 8.25
# f2a = 9.
# f2b = 12.
# f3a = 5.
# f3b = 7.

f1a = 11.
f1b = 13.
f2a = 11.
f2b = 13.
f3a = 11.
f3b = 13.


possibles = will_it_fit(f1a,f1b,f2a,f2b,f3a,f3b)
# possibles = our_fit[0]
# odd = our_fit[1]
# print(possibles[0])

# print('Containers that will fit your space:') 
print('There are',len(possibles), 'containers that will fit your space:')
print('')
for c in possibles:
    print('')
    print(c[0]['title'],c[0]['dimensions'][c[1]])
    print('$' + str(c[0]['price'][c[1]]))
    print(c[0]['url'])


    

There are 14 containers that will fit your space:


White & Black Napa Woven Storage Cube 12" sq. x 12" h
$29.99
https://www.containerstore.com/s/storage/decorative-bins-baskets/white-black-napa-woven-storage-cube/12d?productId=11005708

Patchwork Storage Cube 12" sq. x 12" h
$29.99
https://www.containerstore.com/s/storage/decorative-bins-baskets/patchwork-storage-cube/12d?productId=11001585

Bigso Marten Navy Storage Cube with Leather Handles 12" sq. x 12" h
$19.99
https://www.containerstore.com/s/storage/decorative-bins-baskets/bigso-marten-navy-storage-cube-with-leather-handles/12d?productId=11002268

Bigso Marten Chestnut Storage Cube with Leather Handles 12" sq. x 12" h
$19.99
https://www.containerstore.com/s/storage/decorative-bins-baskets/bigso-marten-chestnut-storage-cube-with-leather-handles/12d?productId=11002269

Bigso Marten Grey Storage Cube with Leather Handles 12" sq. x 12" h
$19.99
https://www.containerstore.com/s/storage/decorative-bins-baskets/bigso-marten-grey-storag

In [340]:
print(data[25]['url'])
print(data[25]['dimensions'])
print(data[25]['new dimensions'])

print(data[26]['dimensions'])
print(data[26]['new dimensions'])
# print(data[27]['dimensions'])
# print(data[27]['new dimensions'])

https://www.containerstore.com/s/storage/decorative-bins-baskets/plastic-wicker-storage-bin-with-handles/12d?productId=11005711
['18" x 12-1/4" x 8" h']
[[18.0, 12.25, 8.0]]
['7" x 11" x 5-1/4" h']
[[7.0, 11.0, 5.25]]


Some of the dimensions give only one or two dimensions.  These are accessories to other storage containers so they are excluded from the will_it_fit function that searches for containers by dimension.

In [342]:
odd = []
for j in range(0,len(data)):
    num_of_containers  = len(data[j]['new dimensions'])
    for k in range(0,num_of_containers):
        if (len(data[j]['new dimensions'][k])!=3):

            odd.append([data[j],k])
    
print('These are some results that are 1 or 2 dimensional divider accessories to containers.  These are excluded from container searches:')    
print('')
for o in odd:
    print('')    
    print(o[0]['title'],o[0]['dimensions'][o[1]])
    print(o[0]['url'])

These are some results that are 1 or 2 dimensional divider accessories to containers.  These are excluded from container searches:


White Wide AkroBin 10-7/8"
https://www.containerstore.com/s/storage/plastic-bins-baskets/white-wide-akrobin/12d?productId=11004565

White Wide AkroBin 18" x 19"
https://www.containerstore.com/s/storage/plastic-bins-baskets/white-wide-akrobin/12d?productId=11004565

White Medium AkroBins 14-3/4"
https://www.containerstore.com/s/storage/plastic-bins-baskets/white-medium-akrobins/12d?productId=11003417

White Medium AkroBins 10-7/8"
https://www.containerstore.com/s/storage/plastic-bins-baskets/white-medium-akrobins/12d?productId=11003417

White Medium AkroBins 18" x 19"
https://www.containerstore.com/s/storage/plastic-bins-baskets/white-medium-akrobins/12d?productId=11003417

White Extra Wide AkroBins 10-7/8"
https://www.containerstore.com/s/storage/plastic-bins-baskets/white-extra-wide-akrobins/12d?productId=11003418

White Extra Wide AkroBins 18" x 19"
htt